In [1]:
# === Schritt 1 ===
# === EV Route Planner Setup ===
# Überprüft, ob alle Libraries funktionieren und erste Tests laufen

# --- Libraries importieren ---
import pandas as pd
import numpy as np
import requests
from sklearn.linear_model import LinearRegression

print("✅ Libraries erfolgreich importiert!")

# --- Beispiel: kleines Test-Dataset ---
data = pd.DataFrame({
    "charging_power": [50, 100, 150, 200],
    "energy_needed": [20, 40, 60, 80],
    "efficiency": [0.9, 0.85, 0.8, 0.75]
})
print("\n📊 Beispiel-Datensatz:")
print(data)

# --- Mini-ML-Test ---
model = LinearRegression()
X = data[["charging_power", "energy_needed", "efficiency"]]
y = [27, 28, 32, 35]  # Beispiel-Zeiten
model.fit(X, y)

prediction = model.predict([[150, 50, 0.85]])
print(f"\n🤖 Beispielhafte Vorhersage: {prediction[0]:.2f} Minuten Ladezeit")


✅ Libraries erfolgreich importiert!

📊 Beispiel-Datensatz:
   charging_power  energy_needed  efficiency
0              50             20        0.90
1             100             40        0.85
2             150             60        0.80
3             200             80        0.75

🤖 Beispielhafte Vorhersage: 31.71 Minuten Ladezeit


/opt/anaconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [5]:
# === Schritt 2 ===
# === MAPBOX API TEST ===
import requests
import pandas as pd

# Dein persönlicher Mapbox Token
MAPBOX_TOKEN = "pk.eyJ1IjoibGludXNkcnAiLCJhIjoiY21odDVlMGh0MXM5bjJrb2htbWxhNWUzbSJ9.51I3gkzO-ruMhsH5lpDKyA"

# Start (Zürich) und Ziel (Genf)
start = "8.5417,47.3769"  # (Lon, Lat)
end = "6.1432,46.2044"

# URL für die Mapbox Directions API
url = f"https://api.mapbox.com/directions/v5/mapbox/driving/{start};{end}?geometries=geojson&access_token={MAPBOX_TOKEN}"

response = requests.get(url)
data = response.json()

# Extrahiere die Koordinaten der Route
route = data["routes"][0]["geometry"]["coordinates"]

print(f"✅ Route erfolgreich abgerufen! Anzahl Punkte: {len(route)}")
print("Erste 5 Punkte:", route[:5])


✅ Route erfolgreich abgerufen! Anzahl Punkte: 42
Erste 5 Punkte: [[8.5417, 47.3769], [8.54186, 47.376887], [8.537505, 47.379624], [8.525312, 47.390411], [8.522141, 47.390968]]


In [7]:
# === Schritt 3 ===
# === OpenChargeMap API Test (mit API-Key) ===

import requests
import pandas as pd

API_KEY = "85078783-26a7-4f9d-b6e5-561d7b87a979"

url = f"https://api.openchargemap.io/v3/poi/?output=json&countrycode=CH&latitude=47.3769&longitude=8.5417&distance=20&distanceunit=KM&key={API_KEY}"

response = requests.get(url)

print("🔎 Statuscode:", response.status_code)
print("🔎 Header Content-Type:", response.headers.get("Content-Type"))
print("🔎 Erste 200 Zeichen der Antwort:")
print(response.text[:200])

chargers = response.json()

print(f"\n🔌 Anzahl gefundener Ladestationen: {len(chargers)}")

# Zeige die ersten 3 Stationen an
for c in chargers[:3]:
    print("----")
    print("Name:", c["AddressInfo"]["Title"])
    print("Latitude:", c["AddressInfo"]["Latitude"])
    print("Longitude:", c["AddressInfo"]["Longitude"])


🔎 Statuscode: 200
🔎 Header Content-Type: application/json
🔎 Erste 200 Zeichen der Antwort:
[
  {
    "DataProvider": {
      "WebsiteURL": "http://www.oplaadpalen.nl/",
      "Comments": null,
      "DataProviderStatusType": {
        "IsProviderEnabled": true,
        "ID": 20,
   

🔌 Anzahl gefundener Ladestationen: 89
----
Name: Seidengasse 15
Latitude: 47.37522
Longitude: 8.53717
----
Name: Jelmoli - the House of Brands
Latitude: 47.373734
Longitude: 8.536409
----
Name: Hotel Glockenhof
Latitude: 47.372876
Longitude: 8.536538


In [10]:
# === Schritt 4 ===
# === Bounding Box für die Route berechnen ===

import numpy as np

# route ist deine Mapbox-Koordinatenliste (lon, lat)
route_array = np.array(route)

min_lon = route_array[:, 0].min()
max_lon = route_array[:, 0].max()
min_lat = route_array[:, 1].min()
max_lat = route_array[:, 1].max()

# kleiner "Puffer" zur Sicherheit (0.05° ~ ca. 5 km)
buffer = 0.05

bbox = {
    "min_lat": min_lat - buffer,
    "max_lat": max_lat + buffer,
    "min_lon": min_lon - buffer,
    "max_lon": max_lon + buffer,
}

print("📦 Bounding Box erfolgreich berechnet:")
print(bbox)



📦 Bounding Box erfolgreich berechnet:
{'min_lat': np.float64(46.153476000000005), 'max_lat': np.float64(47.504506), 'min_lon': np.float64(6.086353), 'max_lon': np.float64(8.59186)}


In [14]:
# === Schritt 5 ===
# === Ladestationen entlang der Route finden ===

import requests

API_KEY = "85078783-26a7-4f9d-b6e5-561d7b87a979"

# Großer Bereich (200 km Radius um CH)
url = (
    f"https://api.openchargemap.io/v3/poi/?output=json"
    f"&countrycode=CH"
    f"&latitude=47.0"
    f"&longitude=8.0"
    f"&distance=200"
    f"&distanceunit=KM"
    f"&key={API_KEY}"
)

response = requests.get(url)
all_chargers = response.json()

print("🔌 Gesamtanzahl CH-Ladestationen abgerufen:", len(all_chargers))

chargers_on_route = []

for c in all_chargers:
    try:
        lat = c["AddressInfo"]["Latitude"]
        lon = c["AddressInfo"]["Longitude"]

        # Filter: nur Charger innerhalb der Bounding Box
        if (
            bbox["min_lat"] <= lat <= bbox["max_lat"]
            and bbox["min_lon"] <= lon <= bbox["max_lon"]
        ):
            chargers_on_route.append(c)

    except Exception as e:
        pass  # Ignoriere Charger ohne Koordinaten

print("🚗🔌 Anzahl Ladestationen entlang der Route:", len(chargers_on_route))

# Beispiele der gefundenen Ladestationen anzeigen
for c in chargers_on_route[:5]:
    print("----")
    print(
        c["AddressInfo"]["Title"],
        c["AddressInfo"]["Latitude"],
        c["AddressInfo"]["Longitude"]
    )


🔌 Gesamtanzahl CH-Ladestationen abgerufen: 100
🚗🔌 Anzahl Ladestationen entlang der Route: 100
----
MOVE - Kambly Trubschachen 46.92050345261245 7.849670267102624
----
Wiederkehr Pneuhaus in Grosswangen 47.1269331 8.0566653
----
Flyer-Factory 47.1162162 7.867689700000028
----
Spycher-Handwerk Ag 47.112009 7.828862
----
Guido A. Zäch Strasse 47.1413 8.129473
